#### Managing the model lifecycle in Unity Catalog

#### 01. Programmatically find the best run and push the model to UC

In [0]:
import mlflow

catalog = 'workspace'
db = 'booking'

booking_experiment_name = "booking_status_binary_mlops_booking_training-2025_07_07-14_35"
model_name = f"{catalog}.{db}.mlops_booking"
print(f"Finding best run from {booking_experiment_name}_* and pushing new model version to {model_name}")
xp_path = f"/Users/{current_user}/databricks_automl"


experiments = mlflow.search_experiments(filter_string=f"name LIKE '{xp_path}/booking_status%'", order_by=["last_update_time DESC"]); experiment_id = experiments[0].experiment_id if experiments else None
print(experiment_id)

#### The best model

In [0]:
# Let's get our best ml run
best_model = mlflow.search_runs(
  experiment_ids=experiment_id,
  order_by=["metrics.test_f1_score DESC"],
  max_results=1,
  filter_string="status = 'FINISHED'" #filter on mlops_best_run to always use the notebook 02 to have a more predictable demo
)
# Optional: Load MLflow Experiment as a spark df and see all runs
# df = spark.read.format("mlflow-experiment").load(experiment_id)
best_model

#### Register to UC

In [0]:
print(f"Registering model to {model_name}")  # {model_name} is defined in the setup script

# Get the run id from the best model
run_id = best_model.iloc[0]['run_id']

# Register the best model from experiments run to MLflow model registry
model_details = mlflow.register_model(f"runs:/{run_id}/model", model_name)

#### Give the registered model a description

In [0]:
from mlflow import MlflowClient

client = MlflowClient()

# The main model description is typically done once.
client.update_registered_model(
  name=model_details.name,
  description="This model predicts whether a customer will cancel their booking using the features in the mlops_booking_training table.",
)

In [0]:
# Provide more details on this specific model version
best_score = best_model['metrics.test_f1_score'].values[0]
run_name = best_model['tags.mlflow.runName'].values[0]
version_desc = f"This model version has an F1 validation metric of {round(best_score,4)*100}%. Follow the link to its training run for more details."

client.update_model_version(
  name=model_details.name,
  version=model_details.version,
  description=version_desc
)

# We can also tag the model version with the F1 score for visibility
client.set_model_version_tag(
  name=model_details.name,
  version=model_details.version,
  key="f1_score",
  value=f"{round(best_score,4)}"
)

#### Set the latest model version as the Challenger model

In [0]:
# Set this version as the Challenger model, using its model alias
client.set_registered_model_alias(
  name=model_name,
  alias="Challenger",
  version=model_details.version
)